In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np



from validation.dot_data import get_dictionary
from toolz import curry
from statsmodels.iolib.tableformatting import fmt_latex, fmt_txt

In [2]:
from validation.scoring import *

In [ ]:
from validation.data import indeed_test_data, dot_train_data, get_soc_n

SOC_LEVEL=3

X_train, y_train = dot_train_data(SOC_LEVEL, include_tasks=False)
count_lookup = y_train.value_counts().to_dict()
code_lookup = make_code_lookup(SOC_LEVEL)

df = pd.read_csv('confusion-matrices/insample-3/sentencespace_100_indeed-lr.csv')
idx = df.sum(1) > 5

df.index = df.columns
test_count = df.sum(1)

scores = [counts(df.values, i) for i,_ in enumerate(df.values)]
sdf = pd.DataFrame({'precision': [prec(tp,fp) for tp,fp,fn in scores], 'recall': [recall(tp,fn) for tp,fp,fn in scores]})


In [11]:
print_tables(format_dfs('recall', sdf, df, idx, code_lookup, count_lookup, test_count), 'recall')        

\begin{subtable}[t]{\linewidth}
\begin{tabular*}{\textwidth}{l @{\extracolsep{\fill}} c c c}
\hline
          Occupation          & Percentage &  DOT/Test & SOC  \\
\hline
\hline
 Other Production Occupations &       0.13 & 4854/4854 & 519  \\
      Material Moving Workers &       0.10 &   598/598 & 537  \\
Textile, Apparel, and Furnis… &       0.10 &   470/470 & 516  \\
      Food Processing Workers &       0.08 &   130/130 & 513  \\
Metal Workers and Plastic Wo… &       0.08 &   808/808 & 514  \\
\hline
\end{tabular*}
\caption{ 519 - (4854/4854) - Other Production Occupations }
\end{subtable}
\vspace{5mm}


\begin{subtable}[t]{\linewidth}
\begin{tabular*}{\textwidth}{l @{\extracolsep{\fill}} c c c}
\hline
          Occupation          & Percentage & DOT/Test & SOC  \\
\hline
\hline
Business Operations Speciali… &       0.22 &  115/115 & 131  \\
               Top Executives &       0.07 &    73/73 & 111  \\
Operations Specialties Manag… &       0.07 &  117/117 & 113  \\
Lawyers, Judg

In [12]:
print_tables(format_dfs('precision', sdf, df, idx, code_lookup, count_lookup, test_count), 'precision')

\begin{subtable}[t]{\linewidth}
\begin{tabular*}{\textwidth}{l @{\extracolsep{\fill}} c c c}
\hline
          Occupation          & Percentage &  DOT/Test & SOC  \\
\hline
\hline
 Other Production Occupations &       0.71 & 4854/4854 & 519  \\
      Food Processing Workers &       0.18 &   130/130 & 513  \\
      Material Moving Workers &       0.04 &   598/598 & 537  \\
Supervisors of Production Wo… &       0.02 &   706/706 & 511  \\
Metal Workers and Plastic Wo… &       0.02 &   808/808 & 514  \\
\hline
\end{tabular*}
\caption{ 513 - (130/130) - Food Processing Workers }
\end{subtable}
\vspace{5mm}


\begin{subtable}[t]{\linewidth}
\begin{tabular*}{\textwidth}{l @{\extracolsep{\fill}} c c c}
\hline
          Occupation          & Percentage &  DOT/Test & SOC  \\
\hline
\hline
 Other Production Occupations &       0.42 & 4854/4854 & 519  \\
 Helpers, Construction Trades &       0.19 &     31/31 & 473  \\
  Construction Trades Workers &       0.15 &   201/201 & 472  \\
Other Installati

In [29]:
f1(*micro(df)), f1(*macro(df, 'macro')), f1(*macro(df, 'weighted'))

(0.4480758680147347, 0.6518370133174584, 0.5242069823358642)

In [31]:
# US SS
f1(*micro(df)), f1(*macro(df, 'macro')), f1(*macro(df, 'weighted'))

(0.4880787295906956, 0.36041481773025075, 0.5159941375489523)

In [ ]:
print_tables(format_dfs('recall', sdf, df, idx), 'recall')        

In [ ]:
print_tables(format_dfs('precision', sdf, df, idx), 'precision')

In [206]:
X_test, y_test, ids = indeed_test_data('../data/us/everything.csv', 100000, 3)

In [207]:
test_data = pd.DataFrame({'X': X_test, 'y': y_test})